# Global indicator project - Phoenix, Arizona

## local neighborhood network accessibility analysis - amenities pois
This notebook uses sausage buffer intersection approach to calculate the accessibility score of sample point local neighborhood. This process use hard threshold distance to count whether any pois (in this case, it is shops with supermarket and convenience stores) located within the local walkable neighborhoods.

### Process:
1. Download or load points of interest (POIs) from OSM
2. Get sample point local neighborhood (sausage buffer)
3. Local neighbothood with at least one amenity pois intersecting
    - the output results are geoseries of local neighborhood buffer polygon
4. Connect the result to sample point dataframe
    - accessibility score is calculated as 1=at least one amenity pois is within the local walkable neighborhood; 0=none of the pois is within the local walkable neighborhood.




In [2]:
from setup_OSMnetwork import *
from setup_local_analysis import *
from config import *

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
# get bounding box from study region boundary shapefile
# get the 10 km buffered study region to consider the edge effects
bbox = get_bufferedstudyregion_bbox(boundary_filepath, crs=crs, to_crs=to_crs, to_latlong=True)

# check the shapefile format, study region boundary shapefile should specify lat lon geometry
# Bounding box should be formatted as a 4 element tuple: (lng_max, lat_min, lng_min, lat_max)
bbox = [bbox[1], bbox[0], bbox[3], bbox[2]]
print(bbox)

[32.414715167745996, -113.44197628074733, 34.13836571274371, -110.93231493458158]


## 1. Download points of interest (POIs) from OSM
What amenities are considered for daily living pois? - [OSMtag](https://taginfo.openstreetmap.org/keys/amenity): shop=supermarket, convenience

In [4]:
pois_gdf = get_osm_pois_gdf(poi_filepath=poi_filepath, shop=shop, bbox=bbox)
pois_gdf.head()

,shop,name,lat,lon,geometry
0,supermarket,Safeway,33.488551,-112.081794,POINT (-112.0817935 33.4885505)
1,convenience,Circle K,33.334805,-111.806096,POINT (-111.8060957 33.3348051)
2,convenience,7-Eleven,33.480013,-112.029747,POINT (-112.0297471 33.4800128)
3,supermarket,Basha's,33.333295,-111.912173,POINT (-111.9121727 33.3332955)
4,supermarket,Fry's Marketplace,33.321899,-111.930303,POINT (-111.9303033 33.3218992)


In [5]:
# define pois points projection
pois_gdf.crs = to_crs
pois_gdf_proj = ox.project_gdf(pois_gdf) # project to UTM for meter measure

## 2. Get sample point local neighborhood (sausage buffer)

In [6]:
# load sample point geodataframe
sample_points = gpd.GeoDataFrame.from_file(samplepoint_stats_shapefile_path)
sample_points.shape

(600, 33)

In [6]:
# load study region projected graph
# we load the projected graph because the sample point geometry is in a projected coordinate system
G_proj = ox.load_graphml(G_proj_filename, folder=OSM_folder)

In [7]:
# create list of sample points to iterate over
point_locations = []

for point in sample_points.geometry: 
    point = (point.x, point.y)
    point_locations = point_locations + [point]

In [ ]:
# iterate over a list of sample points to create sausage buffer as local walkable neighborhood
start = time.time()
task = 'Buffer network for {} sample points'.format(len(point_locations))
sausagebuffers = []
for point in point_locations:
    sausagebuffers.append(create_sausage_buffer_gdf(G_proj, point))
print('Completed task "{}" in {:,.2f} seconds'.format(task,time.time() - start)) 

## 3. POIs point and sausage buffer intersection

In [ ]:
# make sure pois geometry is in the same projection as the sausage buffer geometry
pois_gdf_proj = pois_gdf_proj.to_crs(sausagebuffers[0].crs)
pois_gdf_proj.crs == sausagebuffers[0].crs

In [ ]:
#get pois within distance threshold of sausage buffer
intersection_node_id = get_nodeid_pois_sausagebuffer_intersection(sausagebuffer_gdf=sausagebuffers, pois_gdf=pois_gdf_proj)
intersection_node_id.head()

## 4. Connect the result in sample point dataframe

In [ ]:
# load previously saved sample point dataframe
sample_points_stats = gpd.GeoDataFrame.from_file(samplepoint_stats_shapefile_path)
sample_points_stats.columns

In [ ]:
# merge sample point dataframe and buffer with pois intersection based on node id to get the accessibility distance
sample_points_stats.node_id = sample_points_stats.node_id.astype(int)
sample_points_stats = pd.merge(left=sample_points_stats, right=intersection_node_id, how='left', left_on='node_id', right_on='pois_walkable_node_id', validate="many_to_one")

In [ ]:
sample_points_stats = sample_points_stats.rename(columns={'pois_walkable_node_id':'Baces_shop'})
#fill none value as 0 indicating that the nearest shop is not within walkable network buffer of the nodes/sample point
#fill values more than 0 as 1 indicating that the nearest shop is within walkable network buffer of the nodes/sample point
sample_points_stats['Baces_shop'] = sample_points_stats['Baces_shop'].fillna(0) 
sample_points_stats.loc[sample_points_stats['Baces_shop'] > 0, 'Baces_shop'] = 1
sample_points_stats.columns

In [ ]:
# save sample point stats dataframe
ox.save_gdf_shapefile(sample_points_stats, filename='{studyregion}_urban_sample_points_stats{suffix}'.format(
        studyregion = placename, suffix = suffix), folder=OSM_folder)